_Supervised learning for classification_

In [ ]:
# Import packages
import pickle
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.figure_factory as ff
from sklearn.decomposition import PCA
from scipy.cluster import hierarchy
from IPython.display import display
# Set notebook mode to work in offline
pyo.init_notebook_mode()

### 1.1 Load the dataset

#### 1.1.1 Matrix with observations and features (X)

In scikit-learn, when working with supervised classification tasks, the typical convention is to represent your data in a DataFrame or array-like structure where each row represents an observation (sample) and each column represents a feature (attribute). However, because genomic data usually have way more features (genes) than observations (samples), we use features as rows and observations as columns.

Please load the file and make it into the format as:
- features as rows
- observations as columns
- all row index and column names should be unique.
- all values in the dataframe should be numerical. All metadata should be cleaned.

Please edit the codes below for loading your data.

In [ ]:
# Load the matrix with observation as columns and features (X) as rows
input_path = "/data/projects/240814_Popzhelyazkova_Bremer_Neuropathologie_DNAmArray/analysis/DNAmArray/processed_mVals.csv"
data = pd.read_csv(input_path, sep=",", index_col=0)
# data.drop(columns=['gene_id'], inplace=True)
# data = data.groupby('').mean()
print(data.shape)
print("#### Data index ####")
print(data.index[0:5])
print("#### Data columns ####")
print(data.columns[0:5])

In [ ]:
# If you want to extract a subset of the data for a test run, please use the codes below.
# This line should be commented out for training the real model.
# data = data.head(1000)

#### 1.1.2 Handle Missing Values

In [ ]:
missing_values = data.isnull().sum()
display(missing_values)

In [ ]:
# Decide on the method to handle missing values
# For example, let's fill missing values with the mean of the column
# data = data.fillna(data.mean())

#### 1.1.3 Check the rows with all zeros

In [ ]:
count = len(data[(data == 0).all(axis=1)])
print("Number of rows with all zeros: "+str(count))
if count > 0:
    data = data[~(data == 0).all(axis=1)]
    print(str(count)+" rows are removed.")

#### 1.1.4. Identify outliers using z-scores

In [ ]:
z_scores = np.abs((data - data.mean()) / data.std())
outliers = (z_scores > 3).sum().sort_values(ascending=False)
print(outliers)

In [ ]:
# Decide on the approach to handle outliers
# For example, let's remove rows with outliers
data = data[(z_scores <= 10).all(axis=1)]
print(data.shape)

#### 1.1.4 Load labels (y)

Please make sure the observation names are consistent to the column names in the above matrix (X)

In [ ]:
# Load the named Series with labels (y) as the content and the observation names as names
# labels_path = "inputs/test_labels.csv"
# labels = pd.read_csv(labels_path, sep="\t", index_col=0).squeeze()
labels = pd.DataFrame(data=data.columns, columns=["label"])
labels[['Group', 'sample']] = labels['label'].str.split('.', n=1, expand=True)
# print("#### Labels ####")
# print(labels.iloc[0:5])
display(labels)

In [ ]:
# Check whether all row names are unique
if data.index.duplicated().any():
    print("Row names are not unique.")
# Check whether all column names are unique
if data.columns.duplicated().any():
    print("Column names are not unique.")
# Check whether there is any non-numeric values in data
is_numeric = pd.to_numeric(data.stack(), errors='coerce').notnull().all()
if not is_numeric:
    print("Some values are not numeric.")

In [ ]:
keep = ['ALS',  'Control',  'IBM',  'Multiminicores',  'NMA',  'PM']
labels = labels[labels['Group'].isin(keep)]
data = data[labels["label"].tolist()]
set(labels["Group"].tolist())

# 1.2 Explore the dataset

This section is for understanding your dataset statistically and visually.

### 1.2.1 Description

In [ ]:
display(data.describe())

In [ ]:
print(data.shape)
print(data.info())
# print(data.head())

### 1.2.2 Process the data for visualization

In [ ]:
# Transform the data into log10 scale for further visualization
data_viz = data
# data_viz = data_viz.reset_index()
data_long = pd.melt(data_viz, id_vars=data_viz.columns[0], var_name='sample', value_name='mVal')
# Merging data long table with labels (y)
data_long = pd.merge(data_long, labels, left_on='sample', right_on="sample")

### 1.2.3. PCA

In [ ]:
# Apply PCA to reduce dimensionality to 3 components
pca = PCA(n_components=3)
pca_data = pca.fit_transform(data_viz.transpose())
pca_data = pd.DataFrame(pca_data)
pca_data.columns = ["PC 1", "PC 2", "PC 3"]
pca_data["sample"] = data_viz.columns
pca_data = pd.merge(pca_data, labels, left_on='sample', right_on="label")
fig = px.scatter_3d(pca_data, x='PC 1', y='PC 2', z='PC 3', color='Group', hover_name="label")
fig.show()

In [ ]:
# Create a box plot using Seaborn
sns.boxplot(data=data_viz,
            flierprops={'marker': 'o', 'markerfacecolor': 'gray',
                        'markersize': 2, 'alpha': 0.3})
# Set the title and labels
# plt.title('Box Plot')
plt.xlabel('Sample')
plt.ylabel('M Value')
# Hide x-axis tick labels
plt.xticks([])
# Show the plot
plt.show()

# 1.3 Save variables for next steps

In [ ]:
directory = 'outputs'
if not os.path.exists(directory):
    os.makedirs(directory)
# Save variables to a file
with open('outputs/01_Variables.pkl', 'wb') as file:
    pickle.dump((data, labels), file)